# Part I: Google Colab & Multi-Layer Perceptron

## Workshop em Deep Learning


```
	
	@author: Rafa Felix <rafael.felixalves@adelaide.edu.au>
	Australian Centre for Robotic Vision (ACRV)
	School of Computer Science
	The University of Adelaide
	
	References:
  [1]
	

```

---

### Instituto Federal de Minas Gerais


# Ambiente Google Colab

O Google Colab consiste em um ambiente criado pelo google que possibilita usuários ter acesso a Unidades Gráficas de Processamento (GPUs)


*   [Tutorial: Google Colab](https://colab.research.google.com/notebooks/io.ipynb)
*   [Python and GPU on Google Colab](https://jovianlin.io/pytorch-with-gpu-in-google-colab/)






# Verificando versão do Python

In [1]:
import sys
sys.version

'3.6.3 (default, Oct  3 2017, 21:45:48) \n[GCC 7.2.0]'

# Introdução e Instalação do Pytorch


[Ṕytorch](https://pytorch.org)
```
 Pytorch é um framework opensource utilizado para o desenvolvimento de redes neurais. Este framework possui suporte para operações matriciais e matématicas em geral. Nos últimos anos, Pytoch, tem recebido grande aderência da comunidade científica devido a sua semelhança com Numpy e Matlab. Umas das princiais vantagens da utilização do Pytorch é sua execução dinâmica.
```
---



In [2]:
# pip3: assistente de pacotes do python 3.
# pytorch: framework para desenvolvimento de algoritmos Deep Learning;

# instalação do pytorch
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

#torchvision: framework do pytorch com banco de dados e modelos de deep learning
#             que já são considerados estáveis pela comunidade científica.

# instalação do torchvision
!pip3 install torchvision


    100% |████████████████████████████████| 592.3MB 53.2MB/s 
    100% |████████████████████████████████| 61kB 2.2MB/s 


    100% |████████████████████████████████| 2.0MB 7.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


# Importando e verificando Pytorch e TorchVision

In [0]:
import torch #importando torch
import torch.nn as nn # importando pacote de neural networks nn

# importando pacote de datasets, a este atribuimos o alias dsets
import torchvision.datasets as dsets 

  # importando pacote de data augmentation
import torchvision.transforms as transforms 

# importando pacote de grafos e gradientes
from torch.autograd import Variable

In [0]:
# importando pacote time
import time

# Exemplo 01: Neural Network básica

Neste tutorial iremos desenvolver uma Rede Neural. Neste momento, não desenvolvemos nenhuma convolução, ou deep learning. O nosso objetivo com está etapa é apresentar os frameworks: colab, python, pytorch e torchvision. O presente tutorial foi adaptado [[1](https://colab.research.google.com/drive/1jxUPzMsAkBboHMQtGyfv5M5c7hU8Ss2c#scrollTo=_rnAhgUlIr5t)]


## Initialização dos parâmetros

In [0]:
input_size    = 784   # The image size = 28 x 28 = 784
hidden_size   = 500   # The number of nodes at the hidden layer
num_classes   = 10    # The number of output classes. In this case, from 0 to 9
num_epochs    = 5     # The number of times entire dataset is trained
batch_size    = 100   # The size of input data took for one iteration
learning_rate = 1e-3  # The speed of convergence

# use_cuda é um parametro que utilizamos em nosso código para
# definir onde utilizaremos a GPU ou não. Como estamos utilizando o Google Colab
# podemos utilizar use_cuda como True. Caso você tente executar em uma outra
# plataforma que não possua GPU, utilize está flag como False.
use_cuda = True

## Download do Dataset

In [6]:
# dsets é o alias que atribuimos ao pacote torchvision.datasets

# MNIST consiste em um dataset de caracteres.
# Normalmente o MNIST é o dataset mais utilizado educacionalmente
# em visão computacional;
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

Processing...
Done!


## Carregando o Datasets

```
Uma vez realizada o download do banco de dados, agora podemos carregar o datasets como varíaveis em nosso código.

torch.utils.data.DataLoader é um carregador que possui diversas funcionalidades para datasets.
```

In [0]:

# Para treino, utilizamos a função shuffle = true
# Utilizamos a função shuffle para evitar overfitting
# e adicionar estocasticidade ao processo.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

# Para test, utilizamos a função shuffle = false
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

## Criando a classe Neural Network

```
Nesta etapa, inicializaremos a classe da nossa primeira Neural Network. Apesar de simples, nossa neural network explora algumas camadas e diferentes ativações.
```


In [0]:
# nn.Module permite que nossa Neural Network herde a estrutura de redes neurais do pytorch.

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Inicializando Neural Network

Nesta etapa, iremos inicializar um objeto do tipo Net. Na sequecência, iremos converter nosso grafo da CPU para GPU.

In [0]:
net = Net(input_size, hidden_size, num_classes)


In [0]:

if use_cuda and torch.cuda.is_available():
    net.cuda()

## Função Objetivo & Otimizador

```
A função objetivo, ou loss, avalia as soluções apresentadas pelo classificador a cada epoca, o que indica se as soluções da rede são boas ou ruins. Por sua vez, o otimizador utiliza o feedback fornecido pela funçao objetivo para ajustar os pesos da rede neural.

Função objetivo: Cross Entropy
Otimizador: Adam
```

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

## Treinando a Neural Network

In [12]:
for epoch in range(num_epochs):
    start = time.time()

    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
    runtime = time.time() -  start
    print("Resumo da epoch [%d/%d] runtime: %.4f"%(epoch+1, num_epochs, runtime))
    print("-"*70)

Epoch [1/5], Step [100/600], Loss: 0.2622
Epoch [1/5], Step [200/600], Loss: 0.3291
Epoch [1/5], Step [300/600], Loss: 0.2802
Epoch [1/5], Step [400/600], Loss: 0.3282
Epoch [1/5], Step [500/600], Loss: 0.1610
Epoch [1/5], Step [600/600], Loss: 0.0920
Resumo da epoch [1/5] runtime: 6.1461
----------------------------------------------------------------------
Epoch [2/5], Step [100/600], Loss: 0.1564
Epoch [2/5], Step [200/600], Loss: 0.1247
Epoch [2/5], Step [300/600], Loss: 0.2440
Epoch [2/5], Step [400/600], Loss: 0.0861
Epoch [2/5], Step [500/600], Loss: 0.1068
Epoch [2/5], Step [600/600], Loss: 0.0777
Resumo da epoch [2/5] runtime: 5.9534
----------------------------------------------------------------------
Epoch [3/5], Step [100/600], Loss: 0.1589
Epoch [3/5], Step [200/600], Loss: 0.0940
Epoch [3/5], Step [300/600], Loss: 0.0546
Epoch [3/5], Step [400/600], Loss: 0.0764
Epoch [3/5], Step [500/600], Loss: 0.0813
Epoch [3/5], Step [600/600], Loss: 0.0562
Resumo da epoch [3/5] runt

## Testando a Neural Network


In [13]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    
    if use_cuda and torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    
    
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 97 %


## Salvando o modelo criado

In [0]:
model_fname = "fnn_model.pkl"
torch.save(net.cpu().state_dict(), model_fname)

## Download local do modelo

In [0]:
from google.colab import files
files.download(model_fname)

## Parabéns
